In [95]:
import numpy as np
import pandas as pd

In [96]:
df=pd.read_csv('dsjobs_version3.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          500 non-null    object 
 1   Salary Estimate    500 non-null    object 
 2   Job Description    499 non-null    object 
 3   Rating             500 non-null    float64
 4   Company Name       500 non-null    object 
 5   Location           500 non-null    object 
 6   Headquarters       500 non-null    object 
 7   Size               500 non-null    object 
 8   Founded            500 non-null    int64  
 9   Type of ownership  500 non-null    object 
 10  Industry           500 non-null    object 
 11  Sector             500 non-null    object 
 12  Revenue            500 non-null    object 
 13  Competitors        500 non-null    object 
dtypes: float64(1), int64(1), object(12)
memory usage: 54.8+ KB


In [97]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,"₹ 9,06,841",Functional Analytics Lead\n\nJob details:\nWor...,3.8,Capgemini\n3.8,Bengaluru,"Paris, France",10000+ employees,1967,Company - Public,Enterprise Software & Network Solutions,Information Technology,₹500+ billion (INR),"Accenture, CGI, Sopra Steria"
1,Data Scientist,₹221K - ₹238K,Provide analytical insights into emerging prob...,3.4,PayPal\n3.4,Bengaluru,"San Jose, CA",10000+ employees,1998,Company - Public,Internet,Information Technology,₹500+ billion (INR),"Square, Amazon, Apple"
2,Data Scientist,"₹ 25,38,462",Description\n\nSHIFT: Day Job\n\nSCHEDULE:\n\n...,3.6,Oracle\n3.6,Hyderabad,"Redwood City, CA",10000+ employees,1977,Company - Public,Enterprise Software & Network Solutions,Information Technology,₹500+ billion (INR),"SAP, Salesforce, Microsoft"
3,Data Scientist,"₹ 15,66,457","At Amazon, we strive to be most customer-centr...",4.2,Amazon\n4.2,Hyderabad,"Seattle, WA",10000+ employees,1994,Company - Public,Internet,Information Technology,₹500+ billion (INR),"Google, Microsoft, Walmart"
4,Data Scientist,"₹ 10,83,817",NaN,3.9,Accenture\n3.9,Mumbai,"Dublin, Ireland",10000+ employees,1989,Company - Public,Consulting,Business Services,₹500+ billion (INR),"Cognizant Technology Solutions, EY, McKinsey &..."


## Salary Estimation

In [99]:
df['Salary Estimate'].head()

#Issues 

#1. Remove comma, rupee
#2. Handle range 
#3. Handle /mo

#removes all rows with '-1' as Salary
df = df[df['Salary Estimate'] != '-1']

#replace symbols and comma 
df['Salary Estimate'] = df['Salary Estimate'].str.replace(',', '')
df['Salary Estimate'] = df['Salary Estimate'].str.replace('₹ ', '')
df['Salary Estimate'] = df['Salary Estimate'].str.replace('₹', '')
df['Salary Estimate'] = df['Salary Estimate'].str.replace('K','000')
df['Salary Estimate'].head()

#factor will be 12 if we have monthly salary or else 1; this will be used to calculate annual salary
df['factor'] = np.where(df['Salary Estimate'].str.contains('/mo'),12,1)
df['factor'] = df['factor'].astype(int)

#remove '/mo'
df['Salary Estimate'] = df['Salary Estimate'].str.replace('/mo','')

#handling range values by splitting into min and max 
df['min'] = df['Salary Estimate'].str.split('-').str[0]
df['max'] = df['Salary Estimate'].str.split('-').str[1]

#for those values which are not present in range, min and max will be equal
df['max'] = df['max'].fillna(df['min'])
df['max']=df['max'].astype(int)
df['min']=df['min'].astype(int)

#calculate annual min and max salary by multiplying with factor 
df['Annual Min'] = df['min'] * df['factor']
df['Annual Min'].astype(float)
df['Annual Max'] = df['max']
df['Annual Max'].astype(float)

#calculate avg salary using annual min and max 
df['Avg Salary'] = df['Annual Min'] + df['Annual Max']
df['Avg Salary'] = df['Avg Salary'].astype(float)
df['Avg Salary'] = df['Avg Salary'] / 2

#checking all cols and removing the ones which will not be needed 
df[['min','max','factor','Salary Estimate','Annual Min','Annual Max','Avg Salary']]
df.drop(['min','max','factor','Salary Estimate'],1,inplace=True)

0       ₹ 9,06,841
1    ₹221K - ₹238K
2      ₹ 25,38,462
3      ₹ 15,66,457
4      ₹ 10,83,817
Name: Salary Estimate, dtype: object